## Purpose of script:
#### Reviewing Spark's linear regression implementation
#### Referencing Jose Portilla's "Spark and Python for Big Data with PySpark" course

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [4]:
training = spark.read.format('libsvm').load('../Datasets/sample_linear_regression_data.txt')

In [5]:
training.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [7]:
lr_model = lr.fit(training)

In [8]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lr_model.intercept

0.14228558260358093

In [10]:
training_summary = lr_model.summary

In [11]:
# how much variance is explained by the model
training_summary.r2

0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

10.16309157133015

In [13]:
all_data = spark.read.format('libsvm').load('../Datasets/sample_linear_regression_data.txt')

In [14]:
# snippet on how to split out training and test data
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [16]:
print(train_data.count())
print(test_data.count())

345
156


In [17]:
new_model = lr.fit(train_data)

In [18]:
test_results = new_model.evaluate(test_data)

In [19]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -23.48194204090165|
|  -22.0979999971243|
|-21.023511673168578|
|-20.581047421948636|
|-17.563934426889627|
|-19.578677249083498|
| -16.21730315959043|
| -14.76516761889396|
| -17.65678868655678|
|-16.303795700921963|
| -17.13834101002135|
|-13.104866949731807|
|-15.546188158317474|
|-14.992287715084917|
|-15.329050597653021|
|-13.557114077607242|
|-15.410180088330115|
|-16.268334528999045|
| -10.93899903147951|
|-14.669307856421229|
+-------------------+
only showing top 20 rows



In [20]:
test_results.rootMeanSquaredError

10.049799045371858

In [21]:
# code snippet for prediction
unlabeled_data = test_data.select('features')

In [22]:
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows



In [23]:
predictions = new_model.transform(unlabeled_data)

In [24]:
predictions.show(5)

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -3.254265141700075|
|(10,[0,1,2,3,4,5,...|-1.4128840931986715|
|(10,[0,1,2,3,4,5,...|-1.8139487437507649|
|(10,[0,1,2,3,4,5,...| 1.1787113917340837|
|(10,[0,1,2,3,4,5,...|-1.6043581960741338|
+--------------------+-------------------+
only showing top 5 rows



## 2nd Example - Customer Expenditure

#### Demonstrates how to convert data to Spark features

In [26]:
data = spark.read.csv('../Datasets/Ecommerce_Customers.csv', inferSchema=True, header=True)

In [28]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [32]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [33]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [34]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [35]:
input_cols = ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership']

assembler = VectorAssembler(inputCols=input_cols, outputCol='features')

In [36]:
output = assembler.transform(data)

In [37]:
output.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
+--------------------+
only showing top 5 rows



In [38]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [39]:
final_data = output.select(['Yearly Amount Spent', 'features'])

In [40]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [41]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [42]:
lr_model = lr.fit(train_data)

In [43]:
test_results = lr_model.evaluate(test_data)

In [44]:
test_results.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-5.3445119100968554|
|   8.89059982651105|
|-3.6507257362530936|
|  18.59307079665814|
| -7.933629825284811|
+-------------------+
only showing top 5 rows



In [45]:
test_results.rootMeanSquaredError

10.237043246647575

In [46]:
test_results.r2

0.9804977639476559

In [48]:
# comparing a root mean squared error of 10, to a data mean of 500 (dollars)
# we see that this model is performing well
# which is backed up by a high R-Squared (model captures 98% of variance)
# this is fake generated data, which partially explains such high performance 
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

